# Data Validation Error Format

Version 0.0.1

Jakob Voß [](https://orcid.org/0000-0002-7613-4123) (Verbundzentrale des GBV (VZG))  
July 2, 2025

This document specifies a data format to report validation errors of digital objects.

# 1. Introduction

Data validation is a crucial part of management of data qualitiy and interoperability. Validation is applied in many ways and contexts, for instance input forms and editors with visual feedback or schema languages with formal error reports. The diversity of use cases imply a variety of error results. No common standard exist to express error reports.[1]

The specification of **Data Validation Error Format** has two goals:

-   unify how validation errors are reported by different validators
-   address positions of errors in validated documents

Last but not least the format should help to better separate validation and presentation of validation results, so both can be solved by different applications.

> **Warning**
>
> The format is strictly limited to errors and error positions. Neither does it include other kinds of analysis results such as statistics and summaries of documents, nor does in include details about validation such as test cases, schema rules, and individual constraints. Errors can be linked to additional information with error types but the semantics of these types is out of the scope of this specification.

## 1.1 Overview

<a href="#fig-validation" class="quarto-xref">Figure 1</a> illustrates the validation process with core concepts used in this specification: a **validator** checks whether a **document** conforms to some requirements and returns a list of **errors** in return. Each error can refer to its locations in the document via **positions**.

<figure class=''>

<pre class="mermaid mermaid-js">graph LR
   document --- validator --&gt; errors
   errors -. positions .-&gt; document
   validator(validator)
</pre>

</figure>

Figure 1: Validation process

Every document conforms to a **document model**. For instance JSON documents conforms to the JSON model, and character strings conforms to the model “sequence of characters from a known character set”. Document models come with **encodings** how to express documents in form of documents on a lower level in form. For instance JSON documents can be encoded with JSON syntax as Unicode strings and Unicode strings can be encoded with UTF-8 as sequences of bytes (solid arrows in <a href="#fig-encodings-and-locators" class="quarto-xref">Figure 2</a>).

> **Note**
>
> Eventually all documents are given as digital objects, encoded as sequence of bytes. Encodings using a sequence of characters are also called textual data formats, in contrast to binary data formats.

Error positions are given in form of **locators**, each expressed in a **locator format**. Locator formats refer to sets of document models: for instance [JSON Pointer](https://datatracker.ietf.org/doc/html/rfc6901) refers to JSON, character and line numbers refer to character strings with defined line breaks, and offsets refer to sequences of elements (<a href="#fig-encodings-and-locators" class="quarto-xref">Figure 2</a>).<!-- ^[Unicode strings are sequences as well but its not obvious whether the elements are code points, code units or characters. Line numbers in Unicode are not trivial neither because multiple definitions of line breaks exist.]-->

<figure class=''>

<pre class="mermaid mermaid-js">graph LR
   JSON -- JSON syntax --&gt; Unicode
   Unicode -- UTF-8   --&gt; Bytes
   Unicode[Unicode string]

   jsonpointer(JSON Pointer)
   char(character number)
   line(line number)
   offset

   style jsonpointer fill:#fff,stroke:#fff
   style char fill:#fff,stroke:#fff
   style line fill:#fff,stroke:#fff
   style offset fill:#fff,stroke:#fff

   jsonpointer -.-&gt; JSON   
   char -.-&gt; Unicode
   line -.-&gt; Unicode
   offset -.-&gt; Bytes
</pre>

</figure>

Figure 2: Example of encodings and locator formats

## 1.2 Example

Documents can be invalid compared to document models on many levels. For example the string `{"åå":5}` is valid JSON but it might be invalid if element `åå` is expected to hold a string instead of a number (<a href="#lst-1" class="quarto-xref">Example 1</a>). The error can be located with JSON Pointer in the JSON document and with line and character number of its encoding as Unicode string:

``` json
{
  "message": "Expected string, got number at element /åå",
  "position": { "jsonpointer": "/åå", "line": "1", "char": "7" }
}
```

Example 1: Error in a JSON document

The document could also be invalid at JSON syntax level, for example of the closing `}` is missing (<a href="#lst-2" class="quarto-xref">Example 2</a>). The error is also located by its byte offset:

``` json
{
  "message": "Unexpected end of JSON input at character 8",
  "position": { "line": "1", "char": "8", "offset": "10" }
}
```

Example 2: Error in JSON syntax

A similar document could be invalid on byte level. The following table illustrates the document from <a href="#lst-1" class="quarto-xref">Example 1</a> with ninth byte replaced by a value not allowed in UTF-8. It is common practice to replace such bytes with the Unicode replacement character `U+FFFD` but the resulting Unicode string is invalid JSON syntax still (<a href="#lst-3" class="quarto-xref">Example 3</a>). The example also illustrates another locator format `linecol` to give a character position by line and column.

|  |  |  |  |  |  |  |  |  |  |  |
|---------:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| **Byte** | `7b` | `22` | `c3` | `a5` | `c3` | `a5` | `22` | `3a` | `c0` | `7d` |
| **Code point** | `U+007B` | `U+0022` | `U+00E5` |  | `U+00E5` |  | `U+007B` | `U+0022` | `ERROR⇒` `U+FFFD` | `U+0022` |
| **Character** | `{` | `"` | `å` |  | `å` |  | `"` | `:` | `�` | `}` |

``` json
[
  {
    "level": "warning",
    "message": "Ill-formed UTF-8 byte sequence at offset 8",
    "position": { "line": "1", "char": "", "offset": "8" }
  },
  {
    "level": "error",
    "message": "Expected JSON value at line 1, column 7",
    "position": { "line": "1", "char": "7", "linecol": "1:7", "offset": "8" }
  }
]
```

Example 3: Invalid JSON on multiple levels

## 1.3 Conformance requirements

The key words “MUST”, “MUST NOT”, “REQUIRED”, “SHOULD”, “SHOULD NOT”, “RECOMMENDED”, “NOT RECOMMENDED”, “MAY”, and “OPTIONAL” in this document are to be interpreted as described in BCP 14 ([RFC 2119](https://tools.ietf.org/html/rfc2119) and [RFC 8174](https://tools.ietf.org/html/rfc8174)) when, and only when, they appear in all capitals, as shown here.

Only section <a href="#sec-errors" class="quarto-xref">Section 2</a> to <a href="#sec-locator-formats" class="quarto-xref">Section 4</a>, excluding examples and notes, and the [list of normative references](#normative-references) are normative parts of this specification.

Specific support of Data Validation Error Format by an application depends on two options. Both MUST be documented by applications:

1.  Support either the full format or only support condense form of [positions](#positions) being locator maps
2.  The set of supported [locator formats](#sec-locator-formats)

[1] A notable exception are formats from software development used in unit testing such as [JUnit XML](https://github.com/testmoapp/junitxml).

# 2. Errors

An **Error** is a JSON object with:

-   mandatory field `message` with an **error message**, being a non-empty string

-   optional field `types` with an array of **error types**, each being a non-empty string. Error types can be used for grouping errors and they SHOULD be URIs. Repetitions of identical strings in the same array SHOULD be ignored.

-   optional field `level` with one of the strings `error` or `warning`.

-   optional field `position` with [positions](#positions)

An error is also called **warning** if field `level` has value `warning`.

> **Note**
>
> Language and localization of error messages is out of the scope of this specification.

# 3. Positions

An error can have one or more **positions**. Positions are

-   either a JSON array of [locators](#locators) (detailled form),

-   or a [locator map](:%20#locator-maps) (condense form).

Every locator map can be transformed to an equivalent array of locators. The reverse transformation is not always possible.

## 3.1 Locators

A **locator** is a JSON object with

-   mandatory field `format` with the name of a [locator format](#locator-formats), being a non-empty string
-   mandatory field `value` with the **locator value**, being a string
-   optional field `position` with nested [positions](#positions)

``` json
{ "format": "line", "value": "7" }
```

Example 4: A simple locator indicating the position line (locator format) 7 (locator value)

Nested positions allow to reference locations within nested documents (<a href="#lst-nested-example" class="quarto-xref">Example 5</a>).

``` json
{
  "message": "Invalid value in line 2 of file example.txt in file archive.zip",
  "position": [ {
    "format": "file",
    "value": "archive.zip",
    "position": [ {
      "format": "file",
      "value": "example.txt",
      "position": [ { "format": "line", "value": "2" } ]
    } ]
  } ]
}
```

Example 5: An error in line 2 of file `example.txt` in archive `archive.zip`

## 3.2 Locator maps

A **locator map** is a JSON object that maps names of [locator formats](#sec-locator-formats) to **locator values**.

`{#lst-locator-map .json lst-cap="A simple locator map indicating the position line 7, character 42} { "line": "7", "char": "42" }`

A locator map is equivalent to an array of locators with key and value of the JSON object entries mapped to field `format` and `value` of each locator. An array of locators can be reduced to a locator map by dropping all nested positions and selecting only the first locator of each locator format.

Applications MAY restrict their support of Data Validation Error Format to positions with locator maps. In this case nested positions and multiple positions of same locator format are not supported.

# 4. Locator formats

Each locator format encodes a locator model that refers to a set of document models.

The Data Validation Error Format requires each locator model to have exactely one encoding called its **locator format** to encode locators as Unicode strings.

A **locator format** is a string that identifies a formal language to locate positions or sections in a document. The identifier must start with lowercase letter `a` to `z`, optionally followed by a sequence of lowercase letters, digits `0` to `9` and/or `-`.

Some locator formats (final version of this specification needs to define a registry of locator formats):

| name | locator format | document model |
|---:|----|----|
| `offset` | number (first: 0) | sequence of elements |
| `line` | line number (first: 1) | sequence of lines |
| `char` | character positions (first: 1) | sequence of characters or code points |
| `linecol` | line number and column | sequence of characters with line breaks |
| `file` | POSIX Path | directory tree |
| `jsonpointer` | JSON Pointer | JSON |
| `xpath` | XPath (or a subset) | XML |
| `fq` | format and path | all binary formats supported by [fq](https://github.com/wader/fq?tab=readme-ov-file#fq) (see <a href="#lst-fq" class="quarto-xref">Example 6</a>) |

The locator formats require some more detailled specification. For instance line number depend on a common definition of line breaks, some formats include U+0B, U+0C, U+85, U+2028, U+2029…

``` json
{
  "message": "Timestamp must not be in the future!",
  "position": {
    "fq": "gzip:.members[0].mtime"
  }
}
```

Example 6: Error using fq to locate the internal timestamp of a file in a .gz archive

> **Note**
>
> More candidates of locator formats to be specified:
>
> -   [RFC 5147](https://tools.ietf.org/html/rfc5147) for ranges of lines and characters
> -   [RFC 7111](https://tools.ietf.org/html/rfc7111) for tabular data
> -   Cell addresses like used in spreadsheet software (column uses Hexavigesimalsystem, row uses numbers, e.g. `C4`)
> -   IIIF (section in an image)
> -   RDF graphs (every subset of an RDF graph is another RDF graph)
> -   Subsets of query languages (SQL, SPARQL…)
> -   PDF highlighted text annotations
> -   `id` for data models that refer to elements with an identifier
> -   [PICA Path](https://format.gbv.de/query/picapath)
> -   [MARCspec](https://format.gbv.de/query/marcspec)
> -   …

# 5. References

## 5.1 Normative References

-   Berners-Lee, T. and Fielding, R. and Masinter, L.: *Uniform Resource Identifier (URI): Generic Syntax*. RFC 3986, January 2005, <http://www.rfc-editor.org/info/rfc3986>.

-   Bradner, S.: *Key words for use in RFCs to Indicate Requirement Levels*. BCP 14, RFC 2119, March 1997, <http://www.rfc-editor.org/info/rfc2119>.

-   Bray, T.: *The JavaScript Object Notation (JSON) Data Interchange Format*. RFC 8259, December 2017. <https://tools.ietf.org/html/rfc8259>

-   Leiba, B.: *Ambiguity of Uppercase vs Lowercase in RFC 2119 Key Words*. BCP 14, RFC 8174, May 2017, <http://www.rfc-editor.org/info/rfc8174>.

## 5.2 Informative references

-   [JSON Schema](https://json-schema.org/) schema language

# Appendices

The following information is non-normative.

## JSON Schemas

Error records can be validated with the non-normative JSON Schema [`schema.json`](schema.json) in the specification repository. Rules not covered by the JSON Schema include:

``` json
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "$defs": {
    "locator": {
      "type": "object",
      "properties": {
        "format": {
          "type": "string",
          "pattern": "^[a-z][a-z0-9-]*$"
        },
        "value": {
          "type": "string"
        },
        "position": { "$ref": "#/$defs/position" }
      },
      "required": ["format", "locator"]
    },
    "position": {
      "description": "positions",
      "anyOf": [
        {
          "type": "array",
          "items": { "$ref": "#/$defs/locator" }
        },
        {
          "type": "object",
          "patternProperties": {
            "^[a-z0-9-]+$": {
              "type": "string"
            }
          },
          "additionalProperties": false
        }
      ]
    }
  },
  "properties": {
    "message": {
      "type": "string",
      "minLength": 1,
      "description": "error message"
    },
    "types": {
      "type": "array",
      "items": {
        "type": "string",
        "minLength": 1,
        "description": "identifier of an error type"
      }
    },
    "level": {
      "type": "string",
      "enum": ["error", "warning"],
      "default": "error",
      "description": "error level ('error' or 'warning')"
    },
    "position": { "$ref": "#/$defs/position" }
  },
  "required": ["message"]
}
```

## Changes

This document is managed in a revision control system at <https://github.com/gbv/validation-error-format>, including an [issue tracker](https://github.com/gbv/validation-error-format/issues).

-   **Version 0.0.0**

    Work in progress.

## Acknowledgements

…

## Notes and ideas

Given a JSON document

``` json
{ "authors": "Bob" }
```

An error in field `authors` could be enriched with document content. However document content conforms to document model but it must be encoded in JSON, e.g.:

``` json
{
  "message": "authors must be array",
  "position": [
    {
      "format": "jsonpointer",
      "value": "/authors",
      "document": [{ "format": "string", "value": "bob" }]
    }
  ]
}
```

Eventually this would require a way to encode arbitrary documents in JSON, this is likely a can of worms.

Positions could also have messages, at least for nested positions:

``` json
{
  "message": "Invalid file foo.txt",
  "position": [
    {
      "format": "file",
      "value": "foo.txt",
      "message": "Invalid value in line 2",
      "document": [{ "format": "string", "value": "alice\nbob\n" }],
      "position": { "line": "2" }
    }
  ]
}
```